you didn't need this file. I opened it for my own test

In [1]:
#load lib
%load_ext sql
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, monotonically_increasing_id

In [2]:
import pandas as pd

In [3]:
#create spark session
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

In [4]:
song_data = "data/song_data/*/*/*/*.json"
dfsong = spark.read.json(song_data)
print(dfsong.count())
dfsong.printSchema()

71
root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [5]:
dataLogPath = "data/log_data/*.json"
dflog = spark.read.json(dataLogPath)
print(dflog.count())
dflog.printSchema()

8056
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
dfsong.show(1, truncate=False)

+------------------+---------------+---------------+----------------+----------------------------------------------------------------------------------------------+---------+---------+------------------+-------------------------------+----+
|artist_id         |artist_latitude|artist_location|artist_longitude|artist_name                                                                                   |duration |num_songs|song_id           |title                          |year|
+------------------+---------------+---------------+----------------+----------------------------------------------------------------------------------------------+---------+---------+------------------+-------------------------------+----+
|ARDR4AC1187FB371A1|null           |               |null            |Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti|511.16363|1        |SOBAYLL12A8C138AF9|Sono andati? Fingevo di dormire|0   |
+------------------+---------------+

In [7]:
dflog.show(1, truncate=False)

+--------+---------+---------+------+-------------+--------+---------+-----+----------------------------------+------+--------+-----------------+---------+-------------+------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------+------+
|artist  |auth     |firstName|gender|itemInSession|lastName|length   |level|location                          |method|page    |registration     |sessionId|song         |status|ts           |userAgent                                                                                                                                |userId|
+--------+---------+---------+------+-------------+--------+---------+-----+----------------------------------+------+--------+-----------------+---------+-------------+------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------+

In [8]:
songs_table = dfsong.select('song_id', 'title', 'artist_id', 'year', 'duration').dropDuplicates()

In [9]:
print(songs_table.show(1))

+------------------+-----------------+------------------+----+---------+
|           song_id|            title|         artist_id|year| duration|
+------------------+-----------------+------------------+----+---------+
|SOGOSOV12AF72A285E|¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934|
+------------------+-----------------+------------------+----+---------+
only showing top 1 row

None


In [10]:
songs_table.write.parquet(partitionBy=['year','artist_id'],path='data/out_song_data/song.parquet',mode='overwrite')

In [11]:
artists_table = dfsong.select('artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude').dropDuplicates()

In [12]:
print(artists_table.show(1))

+------------------+---------------+---------------+---------------+----------------+
|         artist_id|    artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+---------------+---------------+---------------+----------------+
|AR3JMC51187B9AE49D|Backstreet Boys|    Orlando, FL|       28.53823|       -81.37739|
+------------------+---------------+---------------+---------------+----------------+
only showing top 1 row

None


In [13]:
artists_table.write.parquet(path='data/out_song_data/artist.parquet',mode='overwrite')

In [14]:
# filter by actions for song plays
dflog =dflog.filter(dflog.page=='NextSong')

In [15]:
user_table= dflog.select('userId', 'firstName', 'lastName', 'gender', 'level').dropDuplicates()

In [16]:
print(user_table.show(1))

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    57|Katherine|     Gay|     F| free|
+------+---------+--------+------+-----+
only showing top 1 row

None


In [17]:
user_table.write.parquet(path='data/out_song_data/user.parquet',mode='overwrite')

In [18]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda ms:datetime.fromtimestamp(ms/1000).strftime('%H:%M:%S'))


In [19]:
dflog = dflog.withColumn('timestamp', get_timestamp(col('ts')))

In [20]:
dflog.show(1, truncate=False)

+--------+---------+---------+------+-------------+--------+---------+-----+----------------------------------+------+--------+-----------------+---------+-------------+------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------+------+---------+
|artist  |auth     |firstName|gender|itemInSession|lastName|length   |level|location                          |method|page    |registration     |sessionId|song         |status|ts           |userAgent                                                                                                                                |userId|timestamp|
+--------+---------+---------+------+-------------+--------+---------+-----+----------------------------------+------+--------+-----------------+---------+-------------+------+-------------+----------------------------------------------------------------------------------------------------------------------

In [21]:
get_datetime= udf(lambda ms:datetime.fromtimestamp(ms/1000).strftime('%Y-%m-%d %H:%M:%S'))
dflog = dflog.withColumn('start_time', get_datetime(col('ts')))

In [22]:
dflog.show(1, truncate=False)

+--------+---------+---------+------+-------------+--------+---------+-----+----------------------------------+------+--------+-----------------+---------+-------------+------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------+------+---------+-------------------+
|artist  |auth     |firstName|gender|itemInSession|lastName|length   |level|location                          |method|page    |registration     |sessionId|song         |status|ts           |userAgent                                                                                                                                |userId|timestamp|start_time         |
+--------+---------+---------+------+-------------+--------+---------+-----+----------------------------------+------+--------+-----------------+---------+-------------+------+-------------+------------------------------------------------------------------------------

In [23]:
time_table = dflog.select("start_time",\
                          hour('start_time').alias('hour'),\
                          dayofmonth('start_time').alias('day'),\
                          weekofyear('start_time').alias('week'),\
                          month('start_time').alias('month'),\
                          year('start_time').alias('year'),\
                          date_format('start_time', 'u').alias('weekday')).dropDuplicates()

In [24]:
time_table.show(2, truncate=False)

+-------------------+----+---+----+-----+----+-------+
|start_time         |hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-15 16:30:00|16  |15 |46  |11   |2018|4      |
|2018-11-21 01:47:20|1   |21 |47  |11   |2018|3      |
+-------------------+----+---+----+-----+----+-------+
only showing top 2 rows



In [25]:
# write time table to parquet files partitioned by year and month
time_table.write.parquet(partitionBy=['year','month'],
                         path='data/out_song_data/time.parquet',
                         mode='overwrite')


In [26]:
# read in song data to use for songplays table
song_df = dfsong

In [27]:
songplays_table = dflog.join(song_df, (dflog.artist==dfsong.artist_name) & (dflog.song==dfsong.title))\
.select('start_time', 'userId', 'level', 'song_id', 'artist_id', 'sessionId', 'location', 'userAgent',\
        month('start_time').alias('month'),year('start_time').alias('year')).withColumn("songplay_id", monotonically_increasing_id())


In [28]:
songplays_table.show(5, truncate=False)

+-------------------+------+-----+------------------+------------------+---------+----------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+-----+----+-----------+
|start_time         |userId|level|song_id           |artist_id         |sessionId|location                          |userAgent                                                                                                                                |month|year|songplay_id|
+-------------------+------+-----+------------------+------------------+---------+----------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+-----+----+-----------+
|2018-11-21 21:56:47|15    |paid |SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|818      |Chicago-Naperville-Elgin, IL-IN-WI|"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5